# Load

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *
from fastai.metrics import error_rate
import pandas as pd
import numpy as np

In [ ]:
np.random.seed(2)
bs = 64
# bs = 16   # uncomment this line if you run out of memory even after clicking Kernel->Restart

In [ ]:
path = Path("/floyd")
path_to_dataset = path/("input/shopee_dataset")
# path_to_home = path/("home")

In [ ]:
df = pd.read_csv(path_to_dataset/'train.csv')
df['filename'] = '/input/shopee_dataset/train/train/' + df.category.map(lambda x: f'{x:02d}') + '/' + df.filename 
df['label'] = df.category
df.to_csv(path/"labels.csv", index=False)

# Train 

In [ ]:
data = ImageDataBunch.from_csv(path, ds_tfms=get_transforms(), size=224, bs=bs).normalize(imagenet_stats)


In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)


In [ ]:
learn.load(path/'home/export_1_34.pkl')
learn = load_learner(path, data=ImageList.from_csv(path, 'labels.csv'))


In [ ]:
# learn.lr_find()
# learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(3)

In [ ]:
# # learn.save('stage-1-34')
# learn.load(path/'home/export_1_34.pkl')
# # learn.data = data
# learn

In [ ]:
# learn.freeze()
# learn.fit_one_cycle(2, max_lr=slice(1e-6,1e-4))

In [ ]:
learn.export(path/'export.pkl')
learn.export(path/'home/export_2_34.pkl')

# Test

In [ ]:
df = pd.read_csv(path_to_dataset/'test.csv')
df['filename'] = '/input/shopee_dataset/test/test/' + df.filename
df.to_csv(path/"predict.csv", index=False)

In [ ]:
learn = load_learner(path, test=ImageList.from_csv(path, 'predict.csv'))

In [ ]:
preds, y, loss = learn.get_preds(ds_type=DatasetType.Test, with_loss=True)

In [ ]:
test_predictions_direct = [data.classes[int(x)] for x in np.argmax(preds, 1)]

df = pd.read_csv(path_to_dataset/'test.csv')
df["predict"] = test_predictions_direct
df.to_csv(path/"home/predict_resnet34_2.csv", index=False)